<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/L_spell_check%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='L')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000099,L,68,682,사업장에서,아파트를 위해,아파트 관리
1,id_0000106,L,68,682,사무실에서,계약의대리,부동산중개
2,id_0000142,L,68,681,사무실에서,비주거용,임대
3,id_0000164,L,68,682,상가사무실에서,빌딩입주자 위해,건물관리서비스
4,id_0000193,L,68,682,사업장에서,아파트를 위해,주거용 아파트 관리서비스
...,...,...,...,...,...,...,...
40135,id_0999952,L,68,682,부동산,중개,부동산중개
40136,id_0999980,L,68,682,사업장에서,고객의요청에의해,부동산중개
40137,id_0999983,L,68,682,사업장에서,예채아파트를,관리
40138,id_0999991,L,68,682,부동산중개소에서,계약및중개등을 통해 수수료받음,부동산 거래중개


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000099,L,68,682,사업장에서 아파트를 위해 아파트 관리
1,id_0000106,L,68,682,사무실에서 계약의대리 부동산중개
2,id_0000142,L,68,681,사무실에서 비주거용 임대
3,id_0000164,L,68,682,상가사무실에서 빌딩입주자 위해 건물관리서비스
4,id_0000193,L,68,682,사업장에서 아파트를 위해 주거용 아파트 관리서비스
...,...,...,...,...,...
40135,id_0999952,L,68,682,부동산 중개 부동산중개
40136,id_0999980,L,68,682,사업장에서 고객의요청에의해 부동산중개
40137,id_0999983,L,68,682,사업장에서 예채아파트를 관리
40138,id_0999991,L,68,682,부동산중개소에서 계약및중개등을 통해 수수료받음 부동산 거래중개


In [8]:
df['digit_3'].value_counts()

682    33212
681     6928
Name: digit_3, dtype: int64

## 맞춤법 교정

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-nvdt7r8y
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-nvdt7r8y
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=f2dc65122bb4fdaf6a2932d16b25a8698b3f355abc8b3531ffba885f213630bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-tw15w0cg/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [11]:
df.to_csv('df_L_spellcheck.csv', index=False, encoding='utf-8-sig')